# Challenge: CRUD and performance

- Connect to Weaviate: Use the provided URL and API key to connect to the Weaviate instance.
- Identify Vectors: Check how many vectors (data points) are stored in the database.
- Perform a Search: Search for concepts you're interested in and filter results by a specific language you understand.
- Vector Search: Perform a vector search to see if you get relevant results.

In [1]:
import requests
import os
import json
import weaviate
from weaviate.embedded import EmbeddedOptions

d:\Vector Databases\vector-databases-certification\env\Lib\site-packages\google\protobuf\runtime_version.py:112: UserWarning: Protobuf gencode version 5.27.2 is older than the runtime version 5.28.0 at grpc_health/v1/health.proto. Please avoid checked-in Protobuf gencode that can be obsolete.
  warnings.warn(


In [2]:
# Load the data into Weaviate
cohere_api_key = os.getenv("COHERE_APIKEY")

waeivate_api_key = os.getenv("WEAVIATE_API_KEY")

auth_config = weaviate.AuthApiKey(api_key="76320a90-53d8-42bc-b41d-678647c6672e")

client = weaviate.Client(
    url="https://cohere-demo.weaviate.network/",
    auth_client_secret=auth_config,
    additional_headers={
        "X-Cohere-Api-Key": cohere_api_key
        #"X-Google-Studio-Api-Key": ai_studio_api_key
    }
)

# Check if client is ready
client.is_ready()
print("Client is ready")

d:\Vector Databases\vector-databases-certification\env\Lib\site-packages\weaviate\__init__.py:143: DeprecationWarning: Dep010: Importing AuthApiKey from weaviate is deprecated. Import AuthApiKey from its module: weaviate.auth
  _Warnings.root_module_import(name, map_[name])
d:\Vector Databases\vector-databases-certification\env\Lib\site-packages\weaviate\warnings.py:162: DeprecationWarning: Dep016: Python client v3 `weaviate.Client(...)` connections and methods are deprecated. Update
            your code to use Python client v4 `weaviate.WeaviateClient` connections and methods.

            For Python Client v4 usage, see: https://weaviate.io/developers/weaviate/client-libraries/python
            For code migration, see: https://weaviate.io/developers/weaviate/client-libraries/python/v3_v4_migration
            
  warnings.warn(


Client is ready


In [6]:
# How many vectors are stored in this database
response = client.query.aggregate("Articles").with_meta_count().do()
print(json.dumps(response, indent=3))

{
   "data": {
      "Aggregate": {
         "Articles": [
            {
               "meta": {
                  "count": 9436199
               }
            }
         ]
      }
   }
}


In [14]:
# Search different types of concepts in the database

concepts = ["climate change", "vacations spot in california", "beach"]

results = []
for concept in concepts:
    query = client.query.get("Articles", ["title", "text", "url", "views", "lang"]).with_near_text({"concepts": concept}).with_limit(2).do()
    print(json.dumps(query, indent=3))

{
   "data": {
      "Get": {
         "Articles": [
            {
               "lang": "en",
               "text": "Climate change threatens people with food and water scarcity, increased flooding, extreme heat, more disease, and economic loss. Human migration and conflict can also be a result. The World Health Organization (WHO) calls climate change the greatest threat to global health in the 21st century. Communities may adapt to climate change through efforts like coastline protection or expanding access to air conditioning, but some impacts are unavoidable. Poorer countries are responsible for a small share of global emissions, yet they have the least ability to adapt and are most vulnerable to climate change.",
               "title": "Climate change",
               "url": "https://en.wikipedia.org/wiki?curid=5042951",
               "views": 3000
            },
            {
               "lang": "de",
               "text": "Klimawandel, auch Klimaver\u00e4nderung, Klima\u

In [15]:
# Vector search with specific confition like lan = en
where = {
    "path": ["lang"],
    "operator": "Equal",
    "valueString": "en"
}

# Search different types of concepts in the database

concepts = ["climate change", "vacations spot in california", "beach"]

results = []
for concept in concepts:
    query = (
        client.query
        .get("Articles", ["title", "text", "url", "views", "lang"])
        .with_near_text({"concepts": concept})
        .with_where(where)
        .with_limit(2)
        .do()
             )
    print(json.dumps(query, indent=3))

{
   "data": {
      "Get": {
         "Articles": [
            {
               "lang": "en",
               "text": "Climate change threatens people with food and water scarcity, increased flooding, extreme heat, more disease, and economic loss. Human migration and conflict can also be a result. The World Health Organization (WHO) calls climate change the greatest threat to global health in the 21st century. Communities may adapt to climate change through efforts like coastline protection or expanding access to air conditioning, but some impacts are unavoidable. Poorer countries are responsible for a small share of global emissions, yet they have the least ability to adapt and are most vulnerable to climate change.",
               "title": "Climate change",
               "url": "https://en.wikipedia.org/wiki?curid=5042951",
               "views": 3000
            },
            {
               "lang": "en",
               "text": "Human activity since the Industrial Revolution, 